# 3D Mesh Analysis
This notebook contains methods for 3D mesh analysis and visualization. It is dependent on the __plotly__ package for data and mesh visualization, and on common scipy packages like __numpy__ for data processing. All geometric features are computed natively without other external dependencies to ensure correctness.


In [1]:
#Setup and check required packages
import plotly
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
print("Plotly: ",plotly.__version__)
import numpy as np
print("NumPy: ",np.__version__)
import matplotlib as mplt
import matplotlib.pyplot as plt
print("Matplotlib: ", mplt.__version__)

import time

Plotly:  3.10.0
NumPy:  1.16.3
Matplotlib:  3.0.3


https://plot.ly/python/

## Action Items:
* Improve geometric landmarking
* Plot of geometric landmarking
* Plot metrics versus post-op month to see age correlation

Extracted data features:

    vertices
    faces
    minX/maxX/rangeX
    maxrange
    vertexNeighbors
    faceNeighbors
    faceAreas
    faceNormals
    vertexNormals
    vertexElevation
    vertexAzimuth
    vertexCurvatures
    vertexDirectionalCurvatures
    leftEarIndex, rightEarIndex, nasionIndex, noseIndex

BEGIN  - Classes and Functions

In [2]:
class Mesh:
    def __init__(self, vertices, faces):
        self.vertices = vertices
        self.faces = faces
        
        timeDebug = False
        
        #start_time = time.time()
        #prev_time = time.time()
        
        #Check that all faces are triangular
        if not sum([1 if len(face)>3 else 0 for face in faces])==0: print('All Triangular: False')
        #Flip X sign
        self.vertices[:,0]*=-1.0
    
        #Swap Y and Z to make movement better in plotly
        tempX = np.copy(self.vertices[:,0])
        tempY = np.copy(self.vertices[:,1])
        tempZ = np.copy(self.vertices[:,2])
        self.vertices[:,1]=tempZ
        self.vertices[:,2]=tempY
    
        #Get max/min ranges
        self.maxX=np.max(self.vertices[:,0])
        self.maxY=np.max(self.vertices[:,1])
        self.maxZ=np.max(self.vertices[:,2])
        self.minX=np.min(self.vertices[:,0])
        self.minY=np.min(self.vertices[:,1])
        self.minZ=np.min(self.vertices[:,2])
        self.rangeX=self.maxX-self.minX
        self.rangeY=self.maxY-self.minY
        self.rangeZ=self.maxZ-self.minZ
        self.maxRange=max(self.rangeX,self.rangeY,self.rangeZ)
        #print("MaxX:",maxX," MaxY:",maxY," MaxZ:",maxZ)
        #print("MinX:",minX," MinY:",minY," MinZ:",minZ)
        
        #range_time = time.time()-prev_time
        #prev_time = time.time()

        
        #Construct list of vertex neighbors for each vertex
        self.vertexNeighbors=[]
        for v_ind in range(0,self.vertices.shape[0]):
            mask=np.any(np.isin(faces,v_ind),1)
            neighbors=np.unique(faces[mask])
            neighbors=np.delete(neighbors,np.argwhere(neighbors==v_ind))
            self.vertexNeighbors.append(neighbors)
            
        #vertNeighbor_time = time.time()-prev_time
        #prev_time = time.time()

            
        #Construct list of face neighbors for each vertex
        self.faceNeighbors=[]
        for v_ind in range(0,self.vertices.shape[0]):
            neighbors=np.nonzero(np.any(np.isin(faces,v_ind),1))
            self.faceNeighbors.append(neighbors)

        #faceNeighbor_time = time.time()-prev_time
        #prev_time = time.time()
            
        #Calculate normal and area of each face
        vA=self.vertices[faces[:,0]]
        vB=self.vertices[faces[:,1]]
        vC=self.vertices[faces[:,2]]
        eU=vB-vA
        eV=vC-vA
        dirVecs=np.cross(eU,eV)
        self.faceAreas=np.linalg.norm(dirVecs,ord=2,axis=1,keepdims=True)
        self.faceNormals=dirVecs/self.faceAreas
        
        #Calculate vertex normals using face normals
        vert_norms_list=[]
        for v_ind in range(0,vertices.shape[0]):
            scaledNorms=self.faceAreas[self.faceNeighbors[v_ind]]*self.faceNormals[self.faceNeighbors[v_ind]]
            weightedSum=np.sum(scaledNorms,axis=0)
            weight=np.linalg.norm(weightedSum)
            if(weight==0):
                vert_norms_list.append(np.zeros(3))
            else:
                vert_norms_list.append(weightedSum/weight)
        self.vertexNormals=np.array(vert_norms_list)

        #normalArea_time = time.time()-prev_time
        #prev_time = time.time()
        
        #Calculate elevation and azimuth at each vertex
        smallVal=0.00000000001
        horiz=np.sqrt(self.vertexNormals[:,0]**2.0+self.vertexNormals[:,1]**2.0)+smallVal
        vert=self.vertexNormals[:,2]-smallVal
        self.vertexElevation=np.arctan(vert/horiz)*180.0/np.pi

        self.vertexAzimuth=np.arctan((self.vertexNormals[:,1])/(self.vertexNormals[:,0]+smallVal))-np.pi/2
        self.vertexAzimuth[self.vertexNormals[:,0]<0]+=np.pi
        self.vertexAzimuth*=(-1.0*180.0/np.pi)

        #angle_time = time.time()-prev_time
        #prev_time = time.time()
        
        # Calculate curvature at each vertex using nearest neighbors
        # https://computergraphics.stackexchange.com/questions/1718/what-is-the-simplest-way-to-compute-principal-curvature-for-a-mesh-triangle

        vert_curv_list=[]
        vert_dir_curv_list=[]

        nonZeroVal=0.00003

        for v_ind in range(0, self.vertices.shape[0]):
            posDif= self.vertices[v_ind]- self.vertices[ self.vertexNeighbors[v_ind]]
            posWeight=np.linalg.norm(posDif,ord=2,axis=1,keepdims=True)
            normDif= self.vertexNormals[v_ind]- self.vertexNormals[ self.vertexNeighbors[v_ind]]

            #curves=(normDif*posDif)/(posWeight*posWeight)
            curves=np.sum(normDif*posDif, axis=1)
            #Normalize by posWeight squared
            curves=curves/(np.square(posWeight)).T
            #Geometric mean of each edge
            curve=np.prod(np.abs(curves))**(1.0/curves.size)
            # Zero curve breaks log analysis, occurs for points with only one/two faces
            if(curve==0.0):
                curve=nonZeroVal
            vert_curv_list.append(curve)

            #Breakdown curvature dimenesion by normalized directional component of each edge
            dirCurves=np.abs((curves.T*posDif)/posWeight)    
            dirCurve=np.prod(np.abs(dirCurves),axis=0)**(1.0/dirCurves.shape[0])
            # Zero curve breaks log analysis
            if(not np.all(dirCurve)):
                dirCurve=[nonZeroVal/3.0,nonZeroVal/3.0,nonZeroVal/3.0]
            vert_dir_curv_list.append(dirCurve)

        self.vertexCurvatures=np.array(vert_curv_list)
        self.vertexDirectionalCurvatures=np.array(vert_dir_curv_list)
        
        #curvature_time = time.time()-prev_time
        #prev_time = time.time()
            
        self.findLandmarks()
    
        #landmark_time = time.time()-prev_time
        #prev_time = time.time()
            
        #total_time = prev_time - start_time
        
        #if(timeDebug):
        #    print("Total:",total_time,", Range:",range_time/total_time,", VertNeighbor:",vertNeighbor_time/total_time,", FaceNeighbor:",faceNeighbor_time/total_time, ", NormalArea:",normalArea_time/total_time,", Angle:",angle_time/total_time,", Curvature:",curvature_time/total_time,", Landmark:",landmark_time/total_time)
    
    def findLandmarks(self):
        self.landmarkVertexColors=np.ones_like(self.vertices)

        #start at Max X vertex, then search up Z
        currInd=np.argmax(self.vertices[:,0])
        currZ=self.vertices[currInd,2]
        self.landmarkVertexColors[currInd,:]=[0.0,1.0,1.0]
        self.landmarkVertexColors[self.vertexNeighbors[currInd],:]*=[0.0,0.0,1.0]

        #print("Start Right Ear:",currInd,", Z:",currZ)
        # DYLAN TODO Turned off for now, often going to top of head
        #while(True):
        #    neighborInd=self.vertexNeighbors[currInd]
        #    neighborPos=self.vertices[neighborInd]
        #    if(np.max(neighborPos[:,2])>currZ):
        #        currInd=neighborInd[np.argmax(neighborPos[:,2])]
        #        currZ=self.vertices[currInd,2]
        #        #vertexColors[currInd,:]*=[0.0,1.0,0.0]
        #    else:
        #        break
        
        #print("Final Right Ear:",currInd,", Z:",currZ)
        self.rightEarIndex=currInd
        self.landmarkVertexColors[currInd,:]=[1.0,1.0,0.0]
        self.landmarkVertexColors[self.vertexNeighbors[currInd],:]*=[1.0,0.0,0.0]

        #start at Min X vertex, then search up Z
        currInd=np.argmin(self.vertices[:,0])
        currZ=self.vertices[currInd,2]
        self.landmarkVertexColors[currInd,:]=[0.0,1.0,1.0]
        self.landmarkVertexColors[self.vertexNeighbors[currInd],:]*=[0.0,0.0,1.0]
        #print("Start Left Ear:",currInd,", Z:",currZ)
        # DYLAN TODO Turned off for now, often going to top of head
        #while(True):
        #    neighborInd=self.vertexNeighbors[currInd]
        #    neighborPos=self.vertices[neighborInd]
        #    if(np.max(neighborPos[:,2])>currZ):
        #        currInd=neighborInd[np.argmax(neighborPos[:,2])]
        #        currZ=self.vertices[currInd,2]
        #        #vertexColors[currInd,:]*=[0.0,1.0,0.0]
        #    else:
        #        break
        #print("Final Left Ear:",currInd,", Z:",currZ)
        self.leftEarIndex=currInd
        self.landmarkVertexColors[currInd,:]=[1.0,1.0,0.0]
        self.landmarkVertexColors[self.vertexNeighbors[currInd],:]*=[1.0,0.0,0.0]

        #start at Max Y vertex less than 0 Z, always move up Z, always move down Y, minimize X distance
        tempVerts = np.copy(self.vertices)
        tempVerts[tempVerts[:,2]>0,1]*=0.0
        currInd=np.argmax(tempVerts[:,1])
        [currX,currY,currZ]=self.vertices[currInd]
        self.landmarkVertexColors[currInd,:]=[0.0,1.0,1.0]
        self.landmarkVertexColors[self.vertexNeighbors[currInd],:]*=[0.0,0.0,1.0]

        #print("Start Nose:",currInd,", X:",currX,", Y:",currY,", Z:",currZ)
        self.noseIndex=currInd
        while(True):
            neighborInd=self.vertexNeighbors[currInd]
            neighborPos=self.vertices[neighborInd]
            #Only larger Z values
            #Z-band robustness: base on average neighbor Z delta
            neighborZDriftAverage=np.average(np.abs(neighborPos[:,2]-currZ))
            neighborInd=neighborInd[neighborPos[:,2]>(currZ+(0.75*neighborZDriftAverage))]
            neighborPos=self.vertices[neighborInd]
            #Only smaller Y values
            neighborInd=neighborInd[neighborPos[:,1]<currY]
            neighborPos=self.vertices[neighborInd]
            if(neighborPos.size==0):
                break
            #Decision: of remaining neighbors, minimize X drift
            neighborXDrift=np.abs(neighborPos[:,0]-currX)
            currInd=neighborInd[np.argmin(neighborXDrift)]
            [currX,currY,currZ]=self.vertices[currInd]
            self.landmarkVertexColors[currInd,:]*=[0.0,1.0,0.0]
        #print("Final Nasion:",currInd,", X:",currX,", Y:",currY,", Z:",currZ)
        self.nasionIndex=currInd
        self.landmarkVertexColors[currInd,:]=[1.0,1.0,0.0]
        self.landmarkVertexColors[self.vertexNeighbors[currInd],:]*=[1.0,0.0,0.0]

        #From nasion, continue to brow ridge (TODO what is formal name?)
        while(True):
            neighborInd=self.vertexNeighbors[currInd]
            neighborPos=self.vertices[neighborInd]
            #Only larger Z values
            #Z-band robustness: base on average neighbor Z delta
            neighborZDriftAverage=np.average(np.abs(neighborPos[:,2]-currZ))
            neighborInd=neighborInd[neighborPos[:,2]>(currZ+(0.75*neighborZDriftAverage))]
            neighborPos=self.vertices[neighborInd]
            #Only greater Y values
            neighborInd=neighborInd[neighborPos[:,1]>currY]
            neighborPos=self.vertices[neighborInd]
            if(neighborPos.size==0):
                break
            #Decision: of remaining neighbors, minimize X drift
            #Test, minimize Y drift instead
            neighborXDrift=np.abs(neighborPos[:,0]-currX)
            neighborYDrift=np.abs(neighborPos[:,1]-currY)
            #currInd=neighborInd[np.argmin(neighborXDrift)]
            #currInd=neighborInd[np.argmin(neighborYDrift)]
            currInd=neighborInd[np.argmin(neighborYDrift+neighborXDrift)]
            [currX,currY,currZ]=self.vertices[currInd]
            self.landmarkVertexColors[currInd,:]*=[0.0,1.0,0.0]
        #print("Final Brow Ridge:",currInd,", X:",currX,", Y:",currY,", Z:",currZ)
        self.browRidgeIndex=currInd
        self.landmarkVertexColors[currInd,:]=[1.0,1.0,0.0]
        self.landmarkVertexColors[self.vertexNeighbors[currInd],:]*=[1.0,0.0,0.0]
        #plot_mesh(vertices,faces,vertexColors,"Ear Points: Blue=[Min|Max X] Red=[Local Max Z]<br>Nose Point: Blue=[Max Y] Red=[Min X Drift, Reduce Y, Increase Z (Banded)]")


In [3]:
#Function to capture NP arrays of vertices and faces

def obj_data_to_Mesh(odata):
    # odata is the string read from an obj file
    vertices = []
    faces = []
    lines = odata.splitlines()   
   
    for line in lines:
        slist = line.split()
        if slist:
            if slist[0] == 'v':
                vertex = list(map(float, slist[1:]))
                vertices.append(vertex)
            elif slist[0] == 'f':
                face = []
                for k in range(1, len(slist)):
                    face.append([int(s) for s in slist[k].replace('//','/').split('/')])
                
                if len(face) > 3: # triangulate the n-polyonal face, n>3
                    faces.extend([[face[0][0]-1, face[k][0]-1, face[k+1][0]-1] for k in range(1, len(face)-1)])
                else:    
                    faces.append([face[j][0]-1 for j in range(len(face))])
            else: pass
    
    
    return Mesh(vertices=np.array(vertices), faces=np.array(faces))    

In [4]:
#plot_hist function: Display a histogram from a dataset and variable name

init_notebook_mode(connected=True)

def plot_hist(dataset,dataname):
    plt.hist(dataset)
    datatitle=dataname+" Histogram"
    plt.title(datatitle)
    plt.xlabel(dataname)
    plt.ylabel("Frequency")
    fig = plt.gcf()
    plotly_fig=tls.mpl_to_plotly(fig)
    iplot(plotly_fig)

In [5]:
#plot_hist_double function: Dispaly two different histograms in the same chart

init_notebook_mode(connected=True)

def plot_hist_double(dataset1,dataname1, dataset2, dataname2):
    minVal = np.min(np.append(dataset1,dataset2))
    maxVal = np.max(np.append(dataset1,dataset2))
    bins = np.linspace(minVal,maxVal,num=10)

    
    plt.hist(dataset1, bins,alpha=0.5,label=dataname1)
    plt.hist(dataset2, bins,alpha=0.5,label=dataname2)
    datatitle=dataname1+" vs "+dataname2+" Histogram"
    plt.title(datatitle)
    plt.legend(loc='upper right')
    #plt.xlabel(dataname)
    plt.ylabel("Frequency")
    fig = plt.gcf()
    plotly_fig=tls.mpl_to_plotly(fig)
    iplot(plotly_fig)

In [6]:
#Display 3D Mesh using plotly
init_notebook_mode(connected=True)

def plot_mesh_custom(mesh,vert_colors,title="3D Mesh"):

    x,y,z=mesh.vertices.T
    I,J,K=mesh.faces.T
    mesh3d=dict(type='mesh3d',
            x=x,
            y=y,
            z=z,
            vertexcolor=vert_colors,
            i=I,
            j=J,
            k=K,
            name='',
            showscale=False
        )
    mesh3d.update(lighting=dict(ambient=0.6,
                             diffuse=0.4,
                             fresnel=0.2,
                             specular=0.0,
                             roughness=0.9),
               lightposition=dict(x=200,
                                 y=200,
                                 z=200))
    layout=dict(title=title,
               font=dict(size=14, color="black"),
               width=750,
               height=750,
               scene=dict(xaxis=dict(visible=True),
                         yaxis=dict(visible=True),
                         zaxis=dict(visible=True),
                         aspectratio=dict(x=mesh.rangeX/mesh.maxRange,
                                         y=mesh.rangeY/mesh.maxRange,
                                         z=mesh.rangeZ/mesh.maxRange
                                         ),
                         camera=dict(eye=dict(x=1.10,y=1.10,z=1.10)),
                         ),
               paper_bgcolor='rgb(235,235,235)',
               margin=dict(t=50)
               )

    fig=go.Figure(data=[mesh3d], layout=layout)
    iplot(fig)

In [7]:
def plot_mesh_landmarks(mesh,title="3D Mesh Landmarks"):
    plot_mesh_custom(mesh,mesh.landmarkVertexColors,title)

In [8]:
#Display 3D Mesh using plotly, with color scale mapped from vertex value intensity

def plot_mesh_colorscale(mesh,intensities,title="3D Mesh",intensityTitle="Intensity"):

    x,y,z=mesh.vertices.T
    I,J,K=mesh.faces.T
    mesh3d=dict(type='mesh3d',
            x=x,
            y=y,
            z=z,
            colorbar=dict(title=intensityTitle),
            colorscale=[[0, 'rgb(0, 0, 255)'],
                      [1, 'rgb(255, 0, 0)']],
              #                      [0.5, 'rgb(0, 255, 0)'], 
                                      #[0.5, 'rgb(127, 127, 127)'],


            intensity=intensities,
            i=I,
            j=J,
            k=K,
            name='',
            showscale=True
        )
    mesh3d.update(lighting=dict(ambient=0.6,
                             diffuse=0.4,
                             fresnel=0.2,
                             specular=0.0,
                             roughness=0.9),
               lightposition=dict(x=200,
                                 y=200,
                                 z=200))
    layout=dict(title=title,
               font=dict(size=14, color="black"),
               width=750,
               height=750,
               scene=dict(xaxis=dict(visible=True),
                         yaxis=dict(visible=True),
                         zaxis=dict(visible=True),
                         aspectratio=dict(x=mesh.rangeX/mesh.maxRange,
                                         y=mesh.rangeY/mesh.maxRange,
                                         z=mesh.rangeZ/mesh.maxRange
                                         ),
                         camera=dict(eye=dict(x=1.10,y=1.10,z=1.10)),
                         ),
               paper_bgcolor='rgb(235,235,235)',
               margin=dict(t=50)
               )

    fig=go.Figure(data=[mesh3d], layout=layout)
    iplot(fig)

In [9]:
#Patient class: keep track of pre- and post-op meshes as well as operation details

class Patient:
    def __init__(self,preOpMesh,postOpMesh,group,patientNumber,operationType,postOpYears):
        self.preOpMesh=preOpMesh
        self.postOpMesh=postOpMesh
        self.group = group
        self.patientNumber = patientNumber
        self.operationType = operationType
        self.postOpYears = postOpYears
        #print("debug")
        #print(len(self.preOpMesh.vertices))
        #print(len(self.postOpMesh.vertices))

In [21]:
#Results class: Extract geometric results from a patient mesh (pre or post op)

class Results:
    def __init__(self,patient,isPre=False):
        
        self.patient = patient
        self.group = patient.group
        self.patientNumber = patient.patientNumber
        self.operationType = patient.operationType
        self.postOpYears = patient.postOpYears
        
        if(isPre):
            self.mesh=self.patient.preOpMesh
        else:
            self.mesh=self.patient.postOpMesh
                
        
        self.vertexResults=dict()
        self.vertexMeans=dict()
        self.vertexSTDs=dict()
        
        self.vertexMetrics=list()
        
        
        self.globalResults=dict()
        self.globalMetrics=list()

        regionFront=self.frontRegionVertices()
        
        self.vertexResults["curvFront"]=self.mesh.vertexCurvatures[regionFront]
        self.vertexResults["curvXFront"]=self.mesh.vertexDirectionalCurvatures[regionFront,0]
        self.vertexResults["curvYFront"]=self.mesh.vertexDirectionalCurvatures[regionFront,1]
        self.vertexResults["curvZFront"]=self.mesh.vertexDirectionalCurvatures[regionFront,2]
        self.vertexResults["curvXPercentFront"]=self.mesh.vertexDirectionalCurvatures[regionFront,0]/self.mesh.vertexCurvatures[regionFront]
        self.vertexResults["curvYPercentFront"]=self.mesh.vertexDirectionalCurvatures[regionFront,1]/self.mesh.vertexCurvatures[regionFront]
        self.vertexResults["curvZPercentFront"]=self.mesh.vertexDirectionalCurvatures[regionFront,2]/self.mesh.vertexCurvatures[regionFront]
        
        self.vertexMetrics.append("curvFront")        
        self.vertexMetrics.append("curvXFront")
        self.vertexMetrics.append("curvYFront")
        self.vertexMetrics.append("curvZFront")
        self.vertexMetrics.append("curvXPercentFront")
        self.vertexMetrics.append("curvYPercentFront")
        self.vertexMetrics.append("curvZPercentFront")

        
        regionBack=self.backRegionVertices()
        
        self.vertexResults["curvBack"]=self.mesh.vertexCurvatures[regionBack]
        self.vertexResults["curvXBack"]=self.mesh.vertexDirectionalCurvatures[regionBack,0]
        self.vertexResults["curvYBack"]=self.mesh.vertexDirectionalCurvatures[regionBack,1]
        self.vertexResults["curvZBack"]=self.mesh.vertexDirectionalCurvatures[regionBack,2]
        self.vertexResults["curvXPercentBack"]=self.mesh.vertexDirectionalCurvatures[regionBack,0]/self.mesh.vertexCurvatures[regionBack]
        self.vertexResults["curvYPercentBack"]=self.mesh.vertexDirectionalCurvatures[regionBack,1]/self.mesh.vertexCurvatures[regionBack]
        self.vertexResults["curvZPercentBack"]=self.mesh.vertexDirectionalCurvatures[regionBack,2]/self.mesh.vertexCurvatures[regionBack]
        
        self.vertexMetrics.append("curvBack")        
        self.vertexMetrics.append("curvXBack")
        self.vertexMetrics.append("curvYBack")
        self.vertexMetrics.append("curvZBack")
        self.vertexMetrics.append("curvXPercentBack")
        self.vertexMetrics.append("curvYPercentBack")
        self.vertexMetrics.append("curvZPercentBack")
   

        for currMetric in self.vertexMetrics:
            self.vertexMeans[currMetric]=np.mean(self.vertexResults[currMetric])
            self.vertexSTDs[currMetric]=np.std(self.vertexResults[currMetric])
            
            
            

        self.globalResults["curvRatio"]=self.vertexMeans["curvFront"]/self.vertexMeans["curvBack"]
        self.globalResults["curvXRatio"]=self.vertexMeans["curvXFront"]/self.vertexMeans["curvXBack"]
        self.globalResults["curvYRatio"]=self.vertexMeans["curvYFront"]/self.vertexMeans["curvYBack"]
        self.globalResults["curvZRatio"]=self.vertexMeans["curvZFront"]/self.vertexMeans["curvZBack"]
        self.globalResults["curvXPercentRatio"]=self.vertexMeans["curvXPercentFront"]/self.vertexMeans["curvXPercentBack"]
        self.globalResults["curvYPercentRatio"]=self.vertexMeans["curvYPercentFront"]/self.vertexMeans["curvYPercentBack"]
        self.globalResults["curvZPercentRatio"]=self.vertexMeans["curvZPercentFront"]/self.vertexMeans["curvZPercentBack"]
        
        self.globalMetrics.append("curvRatio")        
        self.globalMetrics.append("curvXRatio")
        self.globalMetrics.append("curvYRatio")
        self.globalMetrics.append("curvZRatio")
        self.globalMetrics.append("curvXPercentRatio")
        self.globalMetrics.append("curvYPercentRatio")
        self.globalMetrics.append("curvZPercentRatio")


            
        self.globalResults["totalArea"] = np.sum(self.mesh.faceAreas[:])
        
        self.globalResults["regionAreaFront"]=0.0
        for v in np.nditer(np.nonzero(regionFront)):
            self.globalResults["regionAreaFront"]+=(np.sum(self.mesh.faceAreas[self.mesh.faceNeighbors[v]]))
        self.globalResults["regionAreaFrontPercentage"] = self.globalResults["regionAreaFront"] / self.globalResults["totalArea"]
        self.globalResults["regionAreaBack"]=0.0
        for v in np.nditer(np.nonzero(regionBack)):
            self.globalResults["regionAreaBack"]+=(np.sum(self.mesh.faceAreas[self.mesh.faceNeighbors[v]]))
        self.globalResults["regionAreaBackPercentage"] = self.globalResults["regionAreaBack"] / self.globalResults["totalArea"]
        
        self.globalMetrics.append("totalArea")        
        self.globalMetrics.append("regionAreaFront")
        self.globalMetrics.append("regionAreaFrontPercentage")
        self.globalMetrics.append("regionAreaBack")
        self.globalMetrics.append("regionAreaBackPercentage")
        

    def frontRegionVertices(self):
        #Cropping test based on geometric landmarks and normal vector angles (azimuth and elevation)

        #Crop mesh below the plane of the brow ridge and ear tips
        # to reduce noise from face and hair
        brp=self.mesh.vertices[self.mesh.browRidgeIndex]
        rep=self.mesh.vertices[self.mesh.rightEarIndex]
        lep=self.mesh.vertices[self.mesh.leftEarIndex]
        cent=self.mesh.vertices[self.mesh.browRidgeIndex]
        left=self.mesh.vertices[self.mesh.rightEarIndex]
        right=self.mesh.vertices[self.mesh.leftEarIndex]
        planeNormal=np.cross(rep-brp,lep-brp)
        abovePlane=np.sum((self.mesh.vertices-brp)*planeNormal,axis=1)<-5.0

        centerP=np.array([0.0,1.0,0.4])
        rightP=np.array([-1.0,0.0,0.0])
        leftP=np.array([1.0,0.0,0.0])
        planeNormal=np.cross(rightP-centerP,leftP-centerP)
        abovePlaneSimp=np.sum((self.mesh.vertices)*planeNormal,axis=1)>20.0
        
        earY=(rep[1]+lep[1])/2
        maxZInd=np.argmax(self.mesh.vertices[:,2])
        yOfMaxZ=self.mesh.vertices[maxZInd,1]
        
        aboveSimple = self.mesh.vertices[:,2]<0
        #inFront=self.mesh.vertices[:,1]>earY
        inFront=self.mesh.vertices[:,1]>yOfMaxZ

        
        ##DYLAN FOR NOW USE ENTIRE PLANE, NOT JUST ANGLES
        #finalCropIndices=np.logical_and(abovePlane,inFront)
        finalCropIndices=np.logical_and(abovePlaneSimp,inFront)

        
        return finalCropIndices
        
    def backRegionVertices(self):
        #Cropping test based on geometric landmarks and normal vector angles (azimuth and elevation)

        #Crop mesh below the plane of the brow ridge and ear tips
        # to reduce noise from face and hair
        brp=self.mesh.vertices[self.mesh.browRidgeIndex]
        rep=self.mesh.vertices[self.mesh.rightEarIndex]
        lep=self.mesh.vertices[self.mesh.leftEarIndex]
        planeNormal=np.cross(rep-brp,lep-brp)
        abovePlane=np.sum((self.mesh.vertices-brp)*planeNormal,axis=1)<-5.0

        earY=(rep[1]+lep[1])/2
        maxZInd=np.argmax(self.mesh.vertices[:,2])
        yOfMaxZ=self.mesh.vertices[maxZInd,1]
        
        aboveSimple = self.mesh.vertices[:,2]>0
        #inBack=self.mesh.vertices[:,1]<earY
        inBack=self.mesh.vertices[:,1]<yOfMaxZ

        
        
        centerP=np.array([0.0,1.0,0.4])
        rightP=np.array([-1.0,0.0,0.0])
        leftP=np.array([1.0,0.0,0.0])
        planeNormal=np.cross(rightP-centerP,leftP-centerP)
        abovePlaneSimp=np.sum((self.mesh.vertices)*planeNormal,axis=1)>20.0
        
        ##DYLAN FOR NOW USE ENTIRE PLANE, NOT JUST ANGLES
        #finalCropIndices=np.logical_and(abovePlane,inBack)
        finalCropIndices=np.logical_and(abovePlaneSimp,inBack)
        
        return finalCropIndices


In [11]:
#GroupResults class: Extract result statistics for a group of patient scans

class GroupResults:
    def __init__(self,resultList):
        self.resultList=resultList
        self.resultCount=len(resultList)
        
        self.results = dict()
        self.means = dict()
        self.stds = dict()

        self.metrics = list()
        
        self.metrics.append("curvFront")
        self.metrics.append("curvXFront")
        self.metrics.append("curvYFront")
        self.metrics.append("curvZFront")
        self.metrics.append("curvXPercentFront")
        self.metrics.append("curvYPercentFront")
        self.metrics.append("curvZPercentFront")
        self.metrics.append("curvBack")
        self.metrics.append("curvXBack")
        self.metrics.append("curvYBack")
        self.metrics.append("curvZBack")
        self.metrics.append("curvXPercentBack")
        self.metrics.append("curvYPercentBack")
        self.metrics.append("curvZPercentBack")
        
        for currMetric in self.metrics:
            self.results[currMetric]=np.empty(self.resultCount)
        
        for r in range(0,self.resultCount):
            for currMetric in self.metrics:
                (self.results[currMetric])[r]=(resultList[r]).vertexMeans[currMetric]        
        
        for currMetric in self.metrics:
            self.means[currMetric]=np.mean(self.results[currMetric])
            self.stds[currMetric]=np.std(self.results[currMetric])

    
  

In [12]:
#DeltaResults class: Extract result changes from ordered lists of pre and post op results

class DeltaResults:
    def __init__(self,preResultList,postResultList):
        self.preResultList=preResultList
        self.postResultList=postResultList
        self.resultCount=len(self.preResultList)
        
        self.results = dict()
        self.means = dict()
        self.stds = dict()

        self.metrics = list()
        
        self.metrics.append("curvFront")
        self.metrics.append("curvXFront")
        self.metrics.append("curvYFront")
        self.metrics.append("curvZFront")
        self.metrics.append("curvXPercentFront")
        self.metrics.append("curvYPercentFront")
        self.metrics.append("curvZPercentFront")
        self.metrics.append("curvBack")
        self.metrics.append("curvXBack")
        self.metrics.append("curvYBack")
        self.metrics.append("curvZBack")
        self.metrics.append("curvXPercentBack")
        self.metrics.append("curvYPercentBack")
        self.metrics.append("curvZPercentBack")
        
        for currMetric in self.metrics:
            self.results[currMetric]=np.empty(self.resultCount)
        
        for r in range(0,self.resultCount):
            for currMetric in self.metrics:
                (self.results[currMetric])[r]=(postResultList[r]).vertexMeans[currMetric]-(preResultList[r]).vertexMeans[currMetric]        
        
        for currMetric in self.metrics:
            self.means[currMetric]=np.mean(self.results[currMetric])
            self.stds[currMetric]=np.std(self.results[currMetric])

In [23]:
#GroupCompare class: perform Student's T-test on two group results for a given metric

import scipy.stats as stats

class GroupCompare:
    def __init__(self,groupResults1,groupResults2,metric):
        self.metric = metric
        n1=groupResults1.resultCount
        n2=groupResults2.resultCount
        s1=groupResults1.stds[self.metric]
        s2=groupResults2.stds[self.metric]
        self.x1=groupResults1.means[self.metric]
        self.x2=groupResults2.means[self.metric]
        
        sqdn1 = (s1**2)/n1
        sqdn2 = (s2**2)/n2
        
        self.se = np.sqrt( sqdn1 + sqdn2 )
        self.df = ((sqdn1+sqdn2)**2) / (((sqdn1**2)/(n1-1))+((sqdn2**2)/(n2-1)))
        self.t = np.abs((self.x1-self.x2)/self.se)
        self.p = stats.t.sf(self.t,self.df) * 2.0

END  - Classes and Functions

Last Updated: 1/11/20

In [14]:
print("Done loading Multi_Mesh_Definitions")

Done loading Multi_Mesh_Definitions
